In [ ]:
!pip3 uninstall keras-nightly
!pip3 uninstall -y tensorflow
!pip3 install keras==2.1.6
!pip3 install tensorflow==2.2.0
!pip3 install h5py==2.10.0

Found existing installation: tensorflow 2.9.2
Uninstalling tensorflow-2.9.2:
  Successfully uninstalled tensorflow-2.9.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 5.1 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 516.3 MB 2.7 kB/s 
     |████████████████████████████████| 454 kB 64.0 MB/s 
     |████████████████████████████████| 26.0 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 35.3 MB/s 
     |████████████████████████████████| 3.0 MB 42.9 MB/s 
     |████████████████████████████████| 152 kB 61.7 MB/s 
  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.2.0
    Uninstalling cachetools-5.2.0:
      S

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# for loading/processing the images

from tensorflow import keras
from tensorflow.python.keras.preprocessing.image import load_img 
from tensorflow.python.keras.preprocessing.image import img_to_array 
from tensorflow.python.keras.applications.vgg16 import preprocess_input 

# models 
from tensorflow.python.keras.applications.vgg16 import VGG16 
from tensorflow.python.keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle



In [ ]:
#!pip install skimage==1.*
from skimage import data, io

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cargar Fotos

In [ ]:
path = r"/content/drive/MyDrive/DataSetML/train"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
cartas = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the cartas list
            cartas.append(file.name)

print(cartas)

['4498.jpg', '1170.jpg', '6149.jpg', '5866.jpg', '3774.jpg', '1049.jpg', '3689.jpg', '3182.jpg', '6746.jpg', '578.jpg', '4825.jpg', '757.jpg', '904.jpg', '4424.jpg', '6018.jpg', '965.jpg', '1895.jpg', '2793.jpg', '2731.jpg', '6798.jpg', '6675.jpg', '50.jpg', '250.jpg', '1601.jpg', '2178 (1).jpg', '2269.jpg', '4539.jpg', '1481.jpg', '3436.jpg', '5213.jpg', '1224.jpg', '8130.jpg', '1475.jpg', '4436.jpg', '1461.jpg', '3743.jpg', '3309.jpg', '217.jpg', '3004.jpg', '3100.jpg', '5297.jpg', '7348.jpg', '3036.jpg', '7479.jpg', '5513.jpg', '178.jpg', '5416.jpg', '5106.jpg', '200.jpg', '4325.jpg', '2288.jpg', '2474.jpg', '7117.jpg', '4651.jpg', '6845.jpg', '6540.jpg', '6849.jpg', '7151.jpg', '115.jpg', '1608.jpg', '3601.jpg', '1321.jpg', '2013.jpg', '2634.jpg', '3787.jpg', '985.jpg', '3382.jpg', '4954.jpg', '7072.jpg', '4147.jpg', '1720.jpg', '6764.jpg', '7297.jpg', '4457.jpg', '3643.jpg', '3244.jpg', '1671.jpg', '8009.jpg', '2556.jpg', '2922.jpg', '7312.jpg', '6888.jpg', '1847.jpg', '2489.jpg',

In [ ]:
print(len(cartas))

7989


Imagen to Array Numpy

In [ ]:
# load the image as a 224x224 array
img = load_img(cartas[0], target_size=(224,224))
# convert from 'PIL.Image.Image' to numpy array
img = np.array(img)

print(img.shape)
#(224, 224, 3)

(224, 224, 3)


Añade dimensión

In [ ]:
reshaped_img = img.reshape(1,224,224,3)
print(reshaped_img.shape)
#(1, 224, 224, 3)

x = preprocess_input(reshaped_img)

(1, 224, 224, 3)


Cargar vector para clustear imagenes

In [ ]:
# load model
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

553467904/553467096 [==============================] - 5s 0us/step


In [ ]:
features = model.predict(reshaped_img)
print(features.shape)

(1, 4096)


In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

Ahora podemos usar esta función feature_extraction para extraer las características de todas las imágenes y almacenar las características en un diccionario con el nombre de archivo como clave.

In [ ]:
data = {}
p = r"/content/drive/MyDrive/DataSetML/train"

# lop through each image in the dataset
i=0

for carta in cartas:
    # try to extract the features and update the dictionary
    try:
        print(i)
        card = extract_features(carta,model)
        data[carta] = card
        i+=1
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        with open(p,'wb') as file:
            pickle.dump(data,file)
          
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape
#(210, 1, 4096)

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)
feat.shape
#(210, 4096)


Se han truncado las últimas 5000 líneas del flujo de salida.
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176

FileNotFoundError: ignored

In [ ]:
# get the unique labels (from the flower_labels.csv) -> No va
df = pd.read_csv('flower_labels.csv')
label = df['label'].tolist()
unique_labels = list(set(label))

In [ ]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)

print(f"Components before PCA: {feat.shape[1]}")
print(f"Components after PCA: {pca.n_components}")

Components before PCA: 4096
Components after PCA: 100


Hacer grupos de los clusters

In [ ]:
path = r"/content/drive/MyDrive/DataSetML/test"
# change the working directory to the path where the images are located
os.chdir(path)

# this list holds all the image filename
test = []

# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.jpg'):
          # adds only the image files to the cartas list
            test.append(file.name)

print(test)

['3578.jpg', '821.jpg', '7495.jpg', '6562.jpg', '1796.jpg', '4598.jpg', '5692.jpg', '3989.jpg', '4374.jpg', '7696.jpg', '7697.jpg', '6417.jpg', '3223.jpg', '1193.jpg', '5685.jpg', '6818.jpg', '2511.jpg', '6777.jpg', '1622.jpg', '2854.jpg', '605.jpg', '7695.jpg', '4161.jpg', '5914.jpg', '5096.jpg', '4363.jpg', '5082.jpg', '1804.jpg', '4001.jpg', '7736.jpg', '1192.jpg', '6559.jpg', '3421.jpg', '946.jpg', '6203.jpg', '1578.jpg', '2918.jpg', '417.jpg', '7653.jpg', '4158.jpg', '5326.jpg', '1154.jpg', '3541.jpg', '2111.jpg', '5898.jpg', '7082.jpg', '3769.jpg', '2311.jpg', '7057.jpg', '7043.jpg', '4990.jpg', '1181.jpg', '5053.jpg', '7876.jpg', '4614.jpg', '6837.jpg', '3973.jpg', '4991.jpg', '65.jpg', '1816.jpg', '1553.jpg', '4128.jpg', '7810.jpg', '3647.jpg', '6501.jpg', '2954.jpg', '6138.jpg', '5431.jpg', '6528.jpg', '7420.jpg', '4277.jpg', '4061.jpg', '5343.jpg', '1127.jpg', '3724.jpg', '3863.jpg', '4328.jpg', '2176.jpg', '504.jpg', '5988.jpg', '4472.jpg', '8102.jpg', '6305.jpg', '1640.jpg'

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=22)
kmeans.fit(x)

predict = kmeans.predict(test)

# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
skimage.io.imshow(groups[2])

TypeError: ignored